In [1]:
#!/usr/bin/env python
# ------------------------------------------------------------------------------------------------------%
# Created by "Chia E Tungom" 09/12/2020                                                                 %
#       Email:      chemago99@yahoo.com                                                                 %
#       Github:     https://github.com/tungom                                                           %
#-------------------------------------------------------------------------------------------------------%

In [2]:
import random
import math
import numpy as np

In [3]:
#Define number of Companies and Queries NC and NQ respectively

NC, NQ = 3, 7

QE = np.random.uniform(15, 20, NC)

QE

array([15.47464427, 18.38618011, 15.07985129])

In [4]:
# Initializing position and Velocity

bounds = [] # maximum number of adds for a for a company

def InitializeX(N, NC, NQ, BDS):
    """ N: population size (interger number)
        NC: Number of Companies
        NQ: Number of Queries
        BDS: bounds (list with tuples e.g [(a,b),...,(c,d)]). List must be same length with D"""

    return np.array([[np.random.uniform(0, BDS[i], NQ) for i in range(NC)] for j in range(N)])

def InitializeV(N, NC, NQ, BDS):
    """ N: population size (interger number)
        NC: Number of Companies
        NQ: Number of Queries
        BDS: bounds (list with tuples e.g [(a,b),...,(c,d)]). List must be same length with D"""

    return np.array([[np.random.uniform(0, BDS[i], NQ) for i in range(NC)] for j in range(N)])

In [5]:
InitializeX(4, NC, NQ, QE)[0].flatten()

array([15.3347798 ,  4.68609502,  7.30450448, 11.26280345, 13.84345264,
        2.90302111,  2.26810684,  5.97868821,  0.30165423,  0.15115803,
       13.30415378, 12.1234316 , 11.76787015, 11.96468213, 13.37434886,
        7.41872183, 11.13708374, 10.15838408,  9.93812785,  1.15681595,
        2.79539154])

In [6]:
# Define Fitness Function

def Revenue(DV, PPD):
    """DV: is decision varaible matrix
      PPD: is price per display"""
    
    return np.sum(np.multiply(PPD, DV))



def C(BC, QC):
    """ CB: budget"""
    
    nb, nq = 0, 0
    
    for i in range(len(BC[0])):
        if BC[0][i] - BC[1][i] > 0:
            nb += 1
            
    for i in range(len(QC[0])):
        if QC[0][i] - QC[1][i] > 0:
            nq += 1
            
    if nb + nq > 0:
        # F = sum([i/50 for i in cs])   # voilated fitness calculation
        F = - abs(Revenue(DV, PPD)*(1+nv)) # negative abs for maximization
    else:
        F = F(DV, PPD)
    return F, nb + nq

In [7]:
def PGbest(function, nc, x, pbest, gbest):
    
    """ function: function to be optimized
        nc: number of constraints
        x: current position of swarm members
        pbest: list of personal best positions [([position],(fitness, voilations)), ..., ([position],(fitness, voilations)) ]
        gbest: position of best individual """
    x = x.flatten()
    fx = [(i, function(i, nc)) for i in x]           #gives the position and fitness value [(position, (fitness, voilations) )]
    Fis = 0  # flag for feasible solution
    
    for i in range(len(x)):
        
        if fx[i][1][1] == 0 and pbest[i][1][1] == 0:    # compare feasible solutions
            Fis += 1
            
            if fx[i][1][0] < pbest[i][1][0]:
                pbest[i] = fx[i]
                
            if pbest[i][1][0] < gbest[1][0]:
                gbest = pbest[i]
                
        elif fx[i][1][1] == 0 and pbest[i][1][1] > 0:  # replace pbest if infeasible with new feasible solution
            pbest[i] = fx[i]
            
            if Fis == 0 and pbest[i][1][0] < gbest[1][0]:
                gbest = pbest[i]
                
        elif fx[i][1][1] > 0 and pbest[i][1][1] > 0: # compare two infeasible solutions 
            if fx[i][1][0] < pbest[i][1][0]:
                pbest[i] = fx[i]
                
            if Fis == 0 and pbest[i][1][0] < gbest[1][0]:
                gbest = pbest[i]
    
    return pbest, gbest
    

In [8]:
def updateV(vel, x, pbest, gbest, probT, w, c1, c2):
    
    """vel: A list of list with velocities of particles [[velocity],...,[velocity]] 
       x: current position of swarm members
       pbest: list of personal best positions [([position],(fitness, voilations)), ..., ([position],(fitness, voilations)) ]
       gbest: position of best individual 
       probT: turbulance probability
       w: inertia weight in the range [0.3, 0.9]
       c1 and c2: acceleration coefficients in the range [0, 4.0]"""
    
    v = []
    
    for i in range(len(x)):
        v.append([])
        tt = 0
        for j in range(len(x[0])):
            r1 = random.random()
            r2 = random.random()
            
            # Turbulance
            if probT > random.random():
                tt +=1
                v[i].append( round ( (random.uniform(0.1, 0.3)*vel[i][j]) + 
                            (random.uniform(1.5, 2.5) * r1*(pbest[i][0][j] - x[i][j])) + 
                            (random.uniform(1.5, 2.5)* r2* (gbest[0][j] - x[i][j]))  + random.uniform(0,1) ) )
            else:
                v[i].append( round ( (random.uniform(0.1, 0.5)*vel[i][j]) + 
                            (random.uniform(1.5, 2.5) * r1*(pbest[i][0][j] - x[i][j])) + 
                            (random.uniform(1.5, 2.5)* r2* (gbest[0][j] - x[i][j])) ) )
        #if tt > 0 :
        #    print(tt, ": Turbulance recorded for Particle ", i)
            
    return v


def updateX(x, vel, bound):
    """vel: A list of list with velocities of particles [[velocity],...,[velocity]] 
       x: current position of swarm members
       bound: boundary constraints [(a,b),...,(c,d)]"""
    NX = []
    for i in range(len(x)):
        NX.append([])
        
        for j in range(len(x[0])):
            NX[i].append( round (x[i][j] + vel[i][j]) )
            #print(x)
            if NX[i][j] < bound[j][0] or NX[i][j] > bound[j][1]:
                NX[i][j] = round ( random.uniform(bound[j][0],bound[j][1]) )
            
    return NX

In [9]:
def CPSO(function, nc, dimension, bounds, population, runs):
    """ funtion: function to be optimized
        nc: number of constraints
        dimension: dimension of problem
        bounds: upper and lower bound for each dimension
        population: number of particles in the swarm
        runs: number of interations """
    
    positions = InitializeX(population, dimension, bounds)
    velocities = InitializeV(population, dimension, bounds)
    
    c1 = 1.5
    c2 = 0.5
    w = 0.6
    
    i = 0
    
    while i < runs : 
        # Turbulance Operator
        temp =  i /runs
        ProbT = (temp**1.7) - (2.0*temp) + 1 
        #print(ProbTurb)
    
        if i == 0:
            pbest = [(i,function(i, nc)) for i in positions]
            pbest, gbest = PGbest(function, nc, positions, pbest, pbest[0])
        
        velocities = updateV(velocities, positions, pbest, gbest, ProbT, w, c1, c2)
        positions = updateX(positions, velocities, bounds)
        pbest, gbest = PGbest(function, nc, positions, pbest, gbest)
        
        if i%int(runs*0.05) == 0 :
            print("Genaration: ", i, " Function Value: ", gbest[1][0], " Voilated Constraints: ", gbest[1][1])
        
        i += 1
        
        #print(min ([function(i) for i in X]))
        
    return gbest

For C1, found a minima at 

[0.3839749318553818,
  0.5309699037883124,
  0.6870035109228104,
  0.6342181916415011,
  0.7371223220075054,
  0.09089851269384253,
  0.6199743093932358,
  0.5034335150336067,
  0.5713648462285218,
  1.2333271878964789,
  0.23280650979497342,
  0.22671552024457675,
  0.9897380967271239],
 (-33.80134561164435, 0)
 
([0.302912870057118,
  0.9018417105145846,
  0.7948066070528094,
  0.48973420062946027,
  0.3944978343062452,
  0.9983807718835119,
  0.6790318387436395,
  0.514301615752897,
  0.48988704201240507,
  0.2365692376537467,
  0.8412532249093417,
  0.15722398920460634,
  0.1206386715491258],
 (-39.531360475177884, 0))

In [10]:
   
nc = 9
D = 13
bounds = [(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,100),(0,100),(0,100),(0,1)]
pop = 40
Iter = 8400

CPSO(C3, nc, D, bounds, pop, Iter)

NameError: name 'C3' is not defined